In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

def get_nutrient_bounds(gender):
    if gender.lower() == 'male':
        return {
            'Energy (Kcal)': (2200, 3000),
            'Protein (g)': (40, 120),
            'Fat (g)': (50, 97),
            'Carbohydrates (g)': (130, 350),
            'Dietary Fibre (g)': (30, 45),
            'Vitamin A (mu g)': (625, 3000),
            'Vitamin C (mg)': (75, 2000),
            'Vitamin B Complex (mg)': (1.3, 100),
            'Calcium (mg)': (1000, 2500),
            'Iron (mg)': (6, 45),
            'Magnesium (mg)': (330, 350),
            'Potassium (mg)': (2000, 4700)
        }
    else:
        return {
            'Energy (Kcal)': (1800, 2400),
            'Protein (g)': (35, 100),
            'Fat (g)': (45, 77),
            'Carbohydrates (g)': (130, 325),
            'Dietary Fibre (g)': (21, 40),
            'Vitamin A (mu g)': (500, 3000),
            'Vitamin C (mg)': (60, 2000),
            'Vitamin B Complex (mg)': (1.1, 100),
            'Calcium (mg)': (800, 2500),
            'Iron (mg)': (10, 45),
            'Magnesium (mg)': (255, 350),
            'Potassium (mg)': (1600, 3400)
        }

def optimize_diet(budget, gender, excel_file):
    df = pd.read_excel(excel_file, sheet_name='Sheet1')
    foods = df['Ingredient Name'].tolist()
    prices = df['Cost'].values
    nutrient_cols = list(get_nutrient_bounds(gender).keys())
    
    # Filter nutrient columns present in data
    nutrient_cols = [col for col in nutrient_cols if col in df.columns]
    
    bounds_dict = get_nutrient_bounds(gender)
    lower_bounds = np.array([bounds_dict[n][0] for n in nutrient_cols])
    upper_bounds = np.array([bounds_dict[n][1] for n in nutrient_cols])
    
    A = df[nutrient_cols].values.T  # nutrients x foods
    c = prices / 100.0  # cost per gram
    
    # Constraints: A x <= upper_bounds and -A x <= -lower_bounds
    A_ub = np.vstack([A, -A, c])
    b_ub = np.concatenate([upper_bounds, -lower_bounds, [budget]])
    
    bounds = [(0, None)] * len(foods)
    
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
    
    if result.success:
        quantities = result.x * 100  # grams
        total_cost = np.dot(result.x, prices)
        print(f"Optimal diet for {gender} with budget ₹{budget}:")
        for i in np.where(quantities > 1)[0]:
            print(f"{foods[i]:<20}: {quantities[i]:.1f} g (₹{prices[i]:.2f}/100g)")
        print(f"Total daily cost: ₹{total_cost:.2f}")
        
        provided = np.dot(A, result.x)
        print("\nNutrients provided:")
        for n, val, lb, ub in zip(nutrient_cols, provided, lower_bounds, upper_bounds):
            print(f"{n:25}: {val:.2f} (bounds: {lb} - {ub})")
    else:
        print("No feasible solution found. Try increasing budget or relaxing constraints.")

if __name__ == "__main__":
    budget = float(input("Enter your daily food budget in ₹: "))
    gender = input("Enter your gender (male/female): ").strip().lower()
    optimize_diet(budget, gender, 'I.xlsx')

Enter your daily food budget in ₹: 500
Enter your gender (male/female): male
Optimal diet for male with budget ₹500.0:
Finger Millet       : 133.4 g (₹12.00/100g)
Rice                : 189.9 g (₹6.60/100g)
Guava               : 27.2 g (₹10.00/100g)
Coriander Leaves    : 9.6 g (₹2.00/100g)
Garlic              : 41.2 g (₹5.30/100g)
Coconut             : 183.3 g (₹7.80/100g)
Milk                : 388.3 g (₹5.00/100g)
Total daily cost: ₹67.35

Nutrients provided:
Energy (Kcal)            : 2200.00 (bounds: 2200.0 - 3000)
Protein (g)              : 47.91 (bounds: 40.0 - 120)
Fat (g)                  : 97.00 (bounds: 50.0 - 97)
Carbohydrates (g)        : 279.08 (bounds: 130.0 - 350)
Dietary Fibre (g)        : 42.20 (bounds: 30.0 - 45)
Vitamin A (mu g)         : 3000.00 (bounds: 625.0 - 3000)
Vitamin C (mg)           : 75.00 (bounds: 75.0 - 2000)
Vitamin B Complex (mg)   : 13.21 (bounds: 1.3 - 100)
Calcium (mg)             : 1000.00 (bounds: 1000.0 - 2500)
Iron (mg)                : 11.39 (bo